In [1]:
from pyspark.sql import SparkSession as ps
import pyspark.sql.functions as f
from pyspark.sql import Row
from pyspark.sql.window import Window
from datetime import datetime, date



In [2]:
spark = ps.builder.getOrCreate()
spark

23/06/26 10:32:00 WARN Utils: Your hostname, Jessies-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.20.10.2 instead (on interface en0)
23/06/26 10:32:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/26 10:32:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
rdd = spark.sparkContext.parallelize([
    (1, 4., 'GFG1', date(2000, 8, 1), datetime(2000, 8, 1, 12, 0)),
    (2, 8., 'GFG2', date(2000, 6, 2), datetime(2000, 6, 2, 12, 0)),
    (3, 5., 'GFG3', date(2000, 5, 3), datetime(2000, 5, 3, 12, 0))])

rdd_df = spark.createDataFrame(rdd, schema=['a', 'b', 'c', 'd', 'e'])

rdd.collect()

[(1,
  4.0,
  'GFG1',
  datetime.date(2000, 8, 1),
  datetime.datetime(2000, 8, 1, 12, 0)),
 (2,
  8.0,
  'GFG2',
  datetime.date(2000, 6, 2),
  datetime.datetime(2000, 6, 2, 12, 0)),
 (3,
  5.0,
  'GFG3',
  datetime.date(2000, 5, 3),
  datetime.datetime(2000, 5, 3, 12, 0))]

In [4]:
ex_df = spark.createDataFrame([
    Row(a=1, b=4., c='GFG1', d=date(2000, 8, 1),
        e=datetime(2000, 8, 1, 12, 0)),
   
    Row(a=2, b=8., c='GFG2', d=date(2000, 6, 2),
        e=datetime(2000, 6, 2, 12, 0)),
   
    Row(a=4, b=5., c='GFG3', d=date(2000, 5, 3),
        e=datetime(2000, 5, 3, 12, 0))
])

ex_df.show()


23/06/26 10:32:16 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+---+---+----+----------+-------------------+
|  a|  b|   c|         d|                  e|
+---+---+----+----------+-------------------+
|  1|4.0|GFG1|2000-08-01|2000-08-01 12:00:00|
|  2|8.0|GFG2|2000-06-02|2000-06-02 12:00:00|
|  4|5.0|GFG3|2000-05-03|2000-05-03 12:00:00|
+---+---+----+----------+-------------------+



In [5]:
ex_df2 = spark.createDataFrame([
    (1, 4., 'GFG1', date(2000, 8, 1),
     datetime(2000, 8, 1, 12, 0)),
   
    (2, 8., 'GFG2', date(2000, 6, 2),
     datetime(2000, 6, 2, 12, 0)),
   
    (3, 5., 'GFG3', date(2000, 5, 3),
     datetime(2000, 5, 3, 12, 0))
], schema='a long, b double, c string, d date, e timestamp')

ex_df2.show()

+---+---+----+----------+-------------------+
|  a|  b|   c|         d|                  e|
+---+---+----+----------+-------------------+
|  1|4.0|GFG1|2000-08-01|2000-08-01 12:00:00|
|  2|8.0|GFG2|2000-06-02|2000-06-02 12:00:00|
|  3|5.0|GFG3|2000-05-03|2000-05-03 12:00:00|
+---+---+----+----------+-------------------+



In [6]:
df = spark.read.csv("./DisneylandReviews.csv")
df.show()

+---------+------+----------+--------------------+--------------------+-------------------+
|      _c0|   _c1|       _c2|                 _c3|                 _c4|                _c5|
+---------+------+----------+--------------------+--------------------+-------------------+
|Review_ID|Rating|Year_Month|   Reviewer_Location|         Review_Text|             Branch|
|670772142|     4|    2019-4|           Australia|If you've ever be...|Disneyland_HongKong|
|670682799|     4|    2019-5|         Philippines|Its been a while ...|Disneyland_HongKong|
|670623270|     4|    2019-4|United Arab Emirates|Thanks God it was...|Disneyland_HongKong|
|670607911|     4|    2019-4|           Australia|HK Disneyland is ...|Disneyland_HongKong|
|670607296|     4|    2019-4|      United Kingdom|the location is n...|Disneyland_HongKong|
|670591897|     3|    2019-4|           Singapore|Have been to Disn...|Disneyland_HongKong|
|670585330|     5|    2019-4|               India|Great place! Your...|Disneylan

In [7]:
df_new = df.withColumnRenamed('_c0', 'Review_ID'
                              ).withColumnRenamed('_c1', 'Rating'
                              ).withColumnRenamed('_c2', 'Year_Month'
                              ).withColumnRenamed('_c3', 'Reviewer_Location'
                              ).withColumnRenamed('_c4', 'Review_Text'
                              ).withColumnRenamed('_c5', 'Branch')
df_new.show()

+---------+------+----------+--------------------+--------------------+-------------------+
|Review_ID|Rating|Year_Month|   Reviewer_Location|         Review_Text|             Branch|
+---------+------+----------+--------------------+--------------------+-------------------+
|Review_ID|Rating|Year_Month|   Reviewer_Location|         Review_Text|             Branch|
|670772142|     4|    2019-4|           Australia|If you've ever be...|Disneyland_HongKong|
|670682799|     4|    2019-5|         Philippines|Its been a while ...|Disneyland_HongKong|
|670623270|     4|    2019-4|United Arab Emirates|Thanks God it was...|Disneyland_HongKong|
|670607911|     4|    2019-4|           Australia|HK Disneyland is ...|Disneyland_HongKong|
|670607296|     4|    2019-4|      United Kingdom|the location is n...|Disneyland_HongKong|
|670591897|     3|    2019-4|           Singapore|Have been to Disn...|Disneyland_HongKong|
|670585330|     5|    2019-4|               India|Great place! Your...|Disneylan

In [8]:
row = df_new.where(df_new.Review_ID == 'Review_ID')
row.show()

+---------+------+----------+-----------------+-----------+------+
|Review_ID|Rating|Year_Month|Reviewer_Location|Review_Text|Branch|
+---------+------+----------+-----------------+-----------+------+
|Review_ID|Rating|Year_Month|Reviewer_Location|Review_Text|Branch|
+---------+------+----------+-----------------+-----------+------+



In [9]:
df_new = df_new.filter(df_new.Review_ID >= 0)
df_new.show()

+---------+------+----------+--------------------+--------------------+-------------------+
|Review_ID|Rating|Year_Month|   Reviewer_Location|         Review_Text|             Branch|
+---------+------+----------+--------------------+--------------------+-------------------+
|670772142|     4|    2019-4|           Australia|If you've ever be...|Disneyland_HongKong|
|670682799|     4|    2019-5|         Philippines|Its been a while ...|Disneyland_HongKong|
|670623270|     4|    2019-4|United Arab Emirates|Thanks God it was...|Disneyland_HongKong|
|670607911|     4|    2019-4|           Australia|HK Disneyland is ...|Disneyland_HongKong|
|670607296|     4|    2019-4|      United Kingdom|the location is n...|Disneyland_HongKong|
|670591897|     3|    2019-4|           Singapore|Have been to Disn...|Disneyland_HongKong|
|670585330|     5|    2019-4|               India|Great place! Your...|Disneyland_HongKong|
|670574142|     3|    2019-3|            Malaysia|Think of it as an...|Disneylan

In [10]:
df_new = df_new.filter(df_new.Year_Month.startswith('2'))
df_new.show()

+---------+------+----------+--------------------+--------------------+-------------------+
|Review_ID|Rating|Year_Month|   Reviewer_Location|         Review_Text|             Branch|
+---------+------+----------+--------------------+--------------------+-------------------+
|670772142|     4|    2019-4|           Australia|If you've ever be...|Disneyland_HongKong|
|670682799|     4|    2019-5|         Philippines|Its been a while ...|Disneyland_HongKong|
|670623270|     4|    2019-4|United Arab Emirates|Thanks God it was...|Disneyland_HongKong|
|670607911|     4|    2019-4|           Australia|HK Disneyland is ...|Disneyland_HongKong|
|670607296|     4|    2019-4|      United Kingdom|the location is n...|Disneyland_HongKong|
|670591897|     3|    2019-4|           Singapore|Have been to Disn...|Disneyland_HongKong|
|670585330|     5|    2019-4|               India|Great place! Your...|Disneyland_HongKong|
|670574142|     3|    2019-3|            Malaysia|Think of it as an...|Disneylan

In [11]:
simpleData = (("James", "Sales", 3000), \
    ("Michael", "Sales", 4600),  \
    ("Robert", "Sales", 4100),   \
    ("Maria", "Finance", 3000),  \
    ("James", "Sales", 3000),    \
    ("Scott", "Finance", 3300),  \
    ("Jen", "Finance", 3900),    \
    ("Jeff", "Marketing", 3000), \
    ("Kumar", "Marketing", 2000),\
    ("Saif", "Sales", 4100) \
  )
 
columns= ["employee_name", "department", "salary"]
win_df = spark.createDataFrame(data = simpleData, schema = columns)
windowSpec  = Window.partitionBy("department").orderBy("salary")

provide a rank to the result within a window partition.

In [12]:
win_df.withColumn("row_number",f.row_number().over(windowSpec)) \
    .show(truncate=False)

+-------------+----------+------+----------+
|employee_name|department|salary|row_number|
+-------------+----------+------+----------+
|Maria        |Finance   |3000  |1         |
|Scott        |Finance   |3300  |2         |
|Jen          |Finance   |3900  |3         |
|Kumar        |Marketing |2000  |1         |
|Jeff         |Marketing |3000  |2         |
|James        |Sales     |3000  |1         |
|James        |Sales     |3000  |2         |
|Robert       |Sales     |4100  |3         |
|Saif         |Sales     |4100  |4         |
|Michael      |Sales     |4600  |5         |
+-------------+----------+------+----------+



cumulative distribution of values within a window partition.

In [13]:
win_df.withColumn("cume_dist",f.cume_dist().over(windowSpec)) \
   .show()

+-------------+----------+------+------------------+
|employee_name|department|salary|         cume_dist|
+-------------+----------+------+------------------+
|        Maria|   Finance|  3000|0.3333333333333333|
|        Scott|   Finance|  3300|0.6666666666666666|
|          Jen|   Finance|  3900|               1.0|
|        Kumar| Marketing|  2000|               0.5|
|         Jeff| Marketing|  3000|               1.0|
|        James|     Sales|  3000|               0.4|
|        James|     Sales|  3000|               0.4|
|       Robert|     Sales|  4100|               0.8|
|         Saif|     Sales|  4100|               0.8|
|      Michael|     Sales|  4600|               1.0|
+-------------+----------+------+------------------+



In [14]:
df_new.select(f.min(df_new.Year_Month)).show()
df_new.select(f.max(df_new.Year_Month)).show()

+---------------+
|min(Year_Month)|
+---------------+
|        2010-10|
+---------------+

+---------------+
|max(Year_Month)|
+---------------+
|         2019-5|
+---------------+



In [15]:
df_new.select(f.avg(df_new.Rating)).show()

+-----------------+
|      avg(Rating)|
+-----------------+
|4.231101565816747|
+-----------------+



In [16]:
df_new.select(f.max(df_new.Year_Month)).show()
df_new.select(f.min(df_new.Year_Month)).show()

+---------------+
|max(Year_Month)|
+---------------+
|         2019-5|
+---------------+



+---------------+
|min(Year_Month)|
+---------------+
|        2010-10|
+---------------+



In [17]:
df_new.groupBy('Rating').count().collect()

[Row(Rating='3', count=4782),
 Row(Rating='5', count=21908),
 Row(Rating='1', count=1338),
 Row(Rating='4', count=10086),
 Row(Rating='2', count=1929)]

In [18]:
pd_df = df_new.toPandas()
pd_df

,Review_ID,Rating,Year_Month,Reviewer_Location,Review_Text,Branch
0,670772142,4,2019-4,Australia,If you've ever been to Disneyland anywhere you...,Disneyland_HongKong
1,670682799,4,2019-5,Philippines,Its been a while since d last time we visit HK...,Disneyland_HongKong
2,670623270,4,2019-4,United Arab Emirates,Thanks God it wasn t too hot or too humid wh...,Disneyland_HongKong
3,670607911,4,2019-4,Australia,HK Disneyland is a great compact park. Unfortu...,Disneyland_HongKong
4,670607296,4,2019-4,United Kingdom,"the location is not in the city, took around 1...",Disneyland_HongKong
...,...,...,...,...,...,...
40038,92198076,4,2011-1,United Kingdom,Although our pick up was prompt the taxi drive...,Disneyland_Paris
40039,92061774,4,2011-1,Germany,Just returned from a 4 days family trip to Dis...,Disneyland_Paris
40040,91995748,1,2010-12,United Kingdom,We spent the 20 Dec 2010 in the Disney park an...,Disneyland_Paris
40041,91984642,2,2010-12,United Kingdom,Well I was really looking forward to this trip...,Disneyland_Paris


In [19]:
df_new.write.csv('./test.csv')

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/Users/jessiebaron/Documents/Disney-Example-Data/test.csv already exists. Set mode as "overwrite" to overwrite the existing path.

In [23]:
df_new.createOrReplaceTempView("Disney")

spark.sql('SELECT * FROM Disney').show()



+---------+------+----------+--------------------+--------------------+-------------------+
|Review_ID|Rating|Year_Month|   Reviewer_Location|         Review_Text|             Branch|
+---------+------+----------+--------------------+--------------------+-------------------+
|670772142|     4|    2019-4|           Australia|If you've ever be...|Disneyland_HongKong|
|670682799|     4|    2019-5|         Philippines|Its been a while ...|Disneyland_HongKong|
|670623270|     4|    2019-4|United Arab Emirates|Thanks God it was...|Disneyland_HongKong|
|670607911|     4|    2019-4|           Australia|HK Disneyland is ...|Disneyland_HongKong|
|670607296|     4|    2019-4|      United Kingdom|the location is n...|Disneyland_HongKong|
|670591897|     3|    2019-4|           Singapore|Have been to Disn...|Disneyland_HongKong|
|670585330|     5|    2019-4|               India|Great place! Your...|Disneyland_HongKong|
|670574142|     3|    2019-3|            Malaysia|Think of it as an...|Disneylan